<a href="https://colab.research.google.com/github/Zhachory1/MusicNST/blob/master/MusicStyleTransferProjectClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Install Dependencies
print 'Installing dependencies...'
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -q pyfluidsynth
!pip install py-midi pretty_midi

Installing dependencies...


In [2]:
##@title Import Libraries
import numpy as np
import time
import argparse
import warnings
import midi
import pretty_midi

from scipy.optimize import fmin_l_bfgs_b
from scipy.misc import imread, imresize, imsave, fromimage, toimage


from keras.models import Model
from keras.layers import Input
from keras.layers.convolutional import Convolution2D, AveragePooling2D, MaxPooling2D
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.utils.layer_utils import convert_all_kernels_in_model  

#@Title Imports
import os
import copy
import fnmatch
import numpy as np
import time
import argparse
import warnings
import midi
import pretty_midi
import pandas as pd
import collections
import random
import tensorflow as tf

from google.colab import files

# Note sequence to piano roll
from magenta.music.pianoroll_lib import PianorollSequence
import magenta.music.sequences_lib as seq_lib
import magenta.music as mm
from magenta.music.midi_io import note_sequence_to_pretty_midi
import magenta.models.music_vae.data as data
from magenta.protobuf import music_pb2

from keras import optimizers, losses
from keras.models import Model, load_model
from keras.layers import Input
from keras.layers import Dense, Lambda, LSTM, RepeatVector
from keras import backend as K
from keras.utils import Sequence
from keras import optimizers, objectives
from keras import backend, losses, utils
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense, Lambda, LSTM, RepeatVector
from keras.layers.convolutional import Convolution2D, AveragePooling2D, MaxPooling2D, Convolution1D
from keras import backend as K
from keras.utils import Sequence, to_categorical
from keras.utils.data_utils import get_file
from keras.utils.layer_utils import convert_all_kernels_in_model  
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

def play(note_sequence):
  mm.play_sequence(note_sequence, synth=mm.fluidsynth)

def download(note_sequence, filename):
  mm.sequence_proto_to_midi_file(note_sequence, filename)
  files.download(filename)

Using TensorFlow backend.



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4483984266628286195
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 2991451838012244064
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 14655156651526091643
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6412137268
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13162901454224045114
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
]


In [0]:
#@title Load VGG Model 
TF_WEIGHTS_PATH_NO_TOP = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'
TF_19_WEIGHTS_PATH_NO_TOP = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [0]:
#@title Global Params
result_prefix = ""
style_masks = None
content_mask = None
color_mask = None
content_weight = 0.025
style_weight = [1.0]
style_scale =1.0
total_variation_weight = 8.5e-5
num_iter = 100
model = "vgg16"
content_loss_type = 0
rescale_image = "False"
rescale_method ="bilinear"
content_layer ="dense1"
init_image = "content"
pooltype = "max"
preserve_color = "False"
min_improvement = 0.0
pooltype = 1 if pooltype == "ave" else 0
# dimensions of the generated picture.
input_width = input_height = 0
input_WIDTH = input_HEIGHT = 0
aspect_ratio = 2

music_size = 128


In [0]:
#@Title Helper functions and global variables 

################################################################################
#                                 Constants 
################################################################################
BATCH_SIZE = 50
NUM_ITER = 25
DF_FNAME = 'midi_filename'
DF_FEATURES = 'midi_features'
DF_COMPOSER = 'canonical_composer'
DF_COMPOSER_INDEX = 'composer_idx'
DF_SPLIT = "split"
PATH_PREFIX = './MusicNST/midi_files/maestro-v1.0.0/'
INDEX_PATH = PATH_PREFIX + "maestro-v1.0.0.csv"
MAX_DIM = 500
COUNT_CUTTING = 0
NUM_UNIQUE_COMPOSERS = 4
################################################################################
#                    Helper Functions to manipulate input data. 
################################################################################

def new_note_sequence(steps_per_quarter, step_per_split):
  """Helper function to quickly create default NoteSequence proto"""
  qns = music_pb2.NoteSequence()
  qns.quantization_info.steps_per_quarter = steps_per_quarter
  qns.total_quantized_steps = step_per_split
  qns.total_time = 2 * int(step_per_split/steps_per_quarter)
  time_signature = qns.time_signatures.add()
  time_signature.numerator = 4
  time_signature.denominator = 4
  time_signature.time = 0
  tempo = qns.tempos.add()
  tempo.qpm = 120 # quarters per minute
  tempo.time = 0
  return qns
  

def split_note_sequences(note_sequence, bars_per_split=16, steps_per_quarter=4):
  """Splits note_sequences into new note sequence of length bars_per_split
  
  Given a note_sequence, we quantize it to steps_per_quarter notes in a beat. We
  then grab 16*bar_per_split steps into separate note_sequences, assuming that 
  the bars are in 4|4 time. Setting bars_per_split to 2 will return an array of 
  note sequences that are of length 32. 
  
  Args:
    note_sequence:      NoteSequence, input note sequence to split
    bars_per_split:     int, number of bars each output note sequence will have. 
    steps_per_quarter:  int, number of steps in each quarter note.
  Return
    np.array[NoteSequences] - output of note sequences of length 
                              bars_per_split*16
  """
  quantized_seq = mm.quantize_note_sequence(note_sequence, steps_per_quarter)
  # Copy QNS into it's own note_sequences, and clear out its notes. We 
  # are going to iterate over the notes until we hit split boundaries. 
  # Once we hit a boundary, we split any ongoing notes in two, one for
  # the new QNS we have been collecting and one for the new one we will 
  # make. Once the bar is completed, we will createa PianorollSequence
  # from it, and start on the next bar. 
  step_per_split = int(steps_per_quarter*4) * bars_per_split
  split_steps = range(0, quantized_seq.total_quantized_steps, step_per_split)
  current_split = 0
  steps_per_quarter = quantized_seq.quantization_info.steps_per_quarter 
  qns = new_note_sequence(steps_per_quarter, step_per_split)
  pianoroll_seqs = []
  extended_notes = new_note_sequence(steps_per_quarter, step_per_split)
  for note in quantized_seq.notes:
    if note.quantized_start_step >= split_steps[current_split] + step_per_split: 
      # End of split sections. Close up QNS and set up next split.
      pianoroll_seqs.append(copy.deepcopy(qns))
      qns = extended_notes
      extended_notes = new_note_sequence(steps_per_quarter, step_per_split)
      current_split += 1

    if note.quantized_end_step >= split_steps[current_split] + step_per_split:
      # Note extends past this split. Split the note into two notes. Add note
      # from note.start_step to end of current split. Save note from start of 
      # next split to note.end_step
      first_half = qns.notes.add()
      first_half.pitch = note.pitch
      first_half.quantized_start_step = note.quantized_start_step - split_steps[current_split] 
      first_half.quantized_end_step = step_per_split

      second_half = extended_notes.notes.add()
      second_half.pitch = note.pitch
      second_half.quantized_start_step = 0
      second_half.quantized_end_step = note.quantized_end_step - split_steps[current_split] 
    else:
      # This is the normal route, where we just copy the note from the
      # sequence into the new one.
      new_note = qns.notes.add()
      new_note.pitch = note.pitch
      new_note.quantized_start_step = note.quantized_start_step - split_steps[current_split] 
      new_note.quantized_end_step = note.quantized_end_step - split_steps[current_split] 
  pianoroll_seqs.append(copy.deepcopy(qns))

  return pianoroll_seqs

def midi_to_pianoroll(midi_filename):
  """Converts Midi Files into np.arrays and concats them.
  
  The arrays have the following types:
  active, weights, onsets, onset_velocities, active_velocities, offsets, 
  control_changes
  
  Args:
    midi_filename: string, filename
  Returns: 
    np.array - train tensor (33, 128, 7)
  """
  note_seq = mm.midi_file_to_note_sequence(midi_filename)
  split_note_seqs = split_note_sequences(note_seq)
  final_array = []
  for note_seq in split_note_seqs:
    # This outputs (33, 128, 7) It's 33 instead of 32 because it adds an end 
    # token at the end of every sequence.
    pnt = seq_lib.sequence_to_pianoroll(
        note_seq, 2, data.MIN_MIDI_PITCH, data.MAX_MIDI_PITCH)
    # t_list = [pnt.active, pnt.weights, pnt.onsets, 
    #                               pnt.onset_velocities, pnt.active_velocities, 
    #                               pnt.offsets, pnt.control_changes]
    t_list = [pnt.onsets]                                  
    final_array.append(np.concatenate(t_list, axis=-1))
  return np.array(final_array)

def pianoroll_to_notes(pianoroll, opt_midi_file_name=""):
  """Helper to obtain note_seq."""
  note_seq = seq_lib.pianoroll_to_note_sequence(pianoroll, 2, 0)
  if opt_midi_file_name != "":
    download(note_seq, opt_midi_file_name)
  return note_seq

def load_midi_data_from_midi_files(filenames):
  """Loads files.
  Args:
    filenames: list of strings, midi filenames
  Returns: 
    pandas dataframe."""
  tensor_dict = collections.OrderedDict()
  index_array = []
  i = 0
  for mn in filenames:
    tensor_dict[mn]=midi_to_pianoroll(PATH_PREFIX+mn)
    index_array.append(i)
    i=i+1
  return tensor_dict


In [0]:
def preprocess_music_for_vgg(music, load_dims=False):
    global input_width, input_height, input_WIDTH, input_HEIGHT, aspect_ratio

    # Expand the temporal music into a 3 channled tensor to complement VGG input
    temp = np.zeros(music.shape + (3,), dtype=np.uint8)
    temp[:, :, 0] = music
    temp[:, :, 1] = music.copy()
    temp[:, :, 2] = music.copy()

    music = temp

    if load_dims:
        input_WIDTH = music.shape[0]
        input_HEIGHT = music.shape[1]
        aspect_ratio = float(input_HEIGHT) / input_WIDTH

        input_width = music_size
        input_height = int(input_width * aspect_ratio)

    music = imresize(music, (input_width, input_height)).astype('float32')


    music = np.expand_dims(music, axis=0)
    return music

In [0]:
# util function to preserve image color
def original_color_transform(content, generated, mask=None):
    generated = fromimage(toimage(generated, mode='RGB'), mode='YCbCr')  # Convert to YCbCr color space

    if mask is None:
        generated[:, :, 1:] = content[:, :, 1:]  # Generated CbCr = Content CbCr
    else:
        width, height, channels = generated.shape

        for i in range(width):
            for j in range(height):
                if mask[i, j] == 1:
                    generated[i, j, 1:] = content[i, j, 1:]

    generated = fromimage(toimage(generated, mode='YCbCr'), mode='RGB')  # Convert to RGB color space
    return generated

In [0]:
def pooling_func(x):
    if pooltype == 1:
        return AveragePooling2D((2, 2), strides=(2, 2))(x)
    else:
        return MaxPooling2D((2, 2), strides=(2, 2))(x)

In [0]:
def get_inputs(content_img, style_img):
  # Content Model
  # get tensor representations of our images
  content_image = midi_to_pianoroll(content_img)
  content_image = content_image[:1,:,:]
  play(pianoroll_to_notes(content_image))
  style_image = midi_to_pianoroll(style_img)
  style_image = style_image[:1,:,:]
  play(pianoroll_to_notes(style_image))

  combination_image = K.placeholder((1, 257, 128))
  # combine the various images into a single Keras tensor
  input_tensor = K.concatenate([content_image,style_image,
                               combination_image], axis=0)
      
  return input_tensor, (3, 257, 128), combination_image, 3   

In [0]:
def get_model(input_tensor, shape):  
  ip = Input(tensor=input_tensor, batch_shape=shape)

  # build the VGG16 network with our 3 images as input
  x = Convolution2D(64, (3, 3), activation='relu', name='conv1_1', padding='same')(ip)
  x = Convolution2D(64, (3, 3), activation='relu', name='conv1_2', padding='same')(x)
  x = pooling_func(x)

  x = Convolution2D(128, (3, 3), activation='relu', name='conv2_1', padding='same')(x)
  x = Convolution2D(128, (3, 3), activation='relu', name='conv2_2', padding='same')(x)
  x = pooling_func(x)

  x = Convolution2D(256, (3, 3), activation='relu', name='conv3_1', padding='same')(x)
  x = Convolution2D(256, (3, 3), activation='relu', name='conv3_2', padding='same')(x)
  x = Convolution2D(256, (3, 3), activation='relu', name='conv3_3', padding='same')(x)
  if model == "vgg19":
      x = Convolution2D(256, (3, 3), activation='relu', name='conv3_4', padding='same')(x)
  x = pooling_func(x)

  x = Convolution2D(512, (3, 3), activation='relu', name='conv4_1', padding='same')(x)
  x = Convolution2D(512, (3, 3), activation='relu', name='conv4_2', padding='same')(x)
  x = Convolution2D(512, (3, 3), activation='relu', name='conv4_3', padding='same')(x)
  if model == "vgg19":
      x = Convolution2D(512, (3, 3), activation='relu', name='conv4_4', padding='same')(x)
  x = pooling_func(x)

  x = Convolution2D(512, (3, 3), activation='relu', name='conv5_1', padding='same')(x)
  x = Convolution2D(512, (3, 3), activation='relu', name='conv5_2', padding='same')(x)
  x = Convolution2D(512, (3, 3), activation='relu', name='conv5_3', padding='same')(x)
  if model == "vgg19":
      x = Convolution2D(512, (3, 3), activation='relu', name='conv5_4', padding='same')(x)
  x = pooling_func(x)

  return Model(ip, x)

In [0]:
def get_custom_model(input_tensor, shape, use_batchnorm=False, use_dropout=False):
  ip = Input(tensor=input_tensor, batch_shape=shape)
  layer_2 = ip
  if use_batchnorm:
    layer_2 = BatchNormalization(axis=-1)(layer_2)
  layer_2 = Convolution1D(filters=64, kernel_size=32, name="conv1", activation='elu', padding='same')(layer_2)
  if use_dropout:
    layer_2 = Dropout(rate=0.1)(layer_2)
  layer_2 = Convolution1D(filters=16, kernel_size=8, name="conv2", activation='elu', padding='same')(layer_2)
  layer_2 = Convolution1D(filters=1, kernel_size=4, name="conv3", activation='elu', padding='same')(layer_2)
  layer_2 = Lambda(lambda x: backend.squeeze(x, axis=-1))(layer_2)
  dense_1 = Dense(units=256, name="dense1", activation='elu')(layer_2)
  if use_dropout:
    layer_2 = Dropout(rate=0.1)(layer_2)
  dense_1 = Dense(units=32, name="dense2", activation='elu')(dense_1)
  dense_1 = Dense(units=NUM_UNIQUE_COMPOSERS, name="dense3", activation='elu')(dense_1)
  dense_1 = Dense(units=NUM_UNIQUE_COMPOSERS, name="output", activation='softmax')(dense_1)
  return Model(ip, dense_1)

In [0]:
def load_model(vgg_model):
  if model == "vgg19":
      weights = get_file('vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5', TF_19_WEIGHTS_PATH_NO_TOP, cache_subdir='models')
  else:
      weights = get_file('vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5', TF_WEIGHTS_PATH_NO_TOP, cache_subdir='models')

  vgg_model.load_weights(weights)

  if K.backend() == 'tensorflow' and K.image_dim_ordering() == "th":
      warnings.warn('You are using the TensorFlow backend, yet you '
                    'are using the Theano '
                    'image dimension ordering convention '
                    '(`image_dim_ordering="th"`). '
                    'For best performance, set '
                    '`image_dim_ordering="tf"` in '
                    'your Keras config '
                    'at ~/.keras/keras.json.')
      convert_all_kernels_in_model(vgg_model)
  print('Model loaded.')

In [0]:
def get_output_dict_and_shape(vgg_model):
  # get the symbolic outputs of each "key" layer (we gave them unique names).
  outputs_dict = dict([(layer.name, layer.output) for layer in vgg_model.layers])
  shape_dict = dict([(layer.name, layer.output_shape) for layer in vgg_model.layers])
  return outputs_dict, shape_dict

In [0]:
# the gram matrix of an image tensor (feature-wise outer product)
def gram_matrix(x):
#     assert K.ndim(x) == 3
    features = x
    if K.ndim(x) == 3:
      if K.image_dim_ordering() == "th":
          features = K.batch_flatten(x)
      else:
          features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    if K.ndim(x) == 2:
      return K.dot(features, K.transpose(features))
    return features * features

In [0]:
# the "style loss" is designed to maintain
# the style of the reference image in the generated image.
# It is based on the gram matrices (which capture style) of
# feature maps from the style reference image
# and from the generated image
def style_loss(style, combination, mask_path=None, nb_channels=None):
#     assert K.ndim(style) == 3
#     assert K.ndim(combination) == 3

    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = input_width * input_height
    return K.sum(K.square(S - C)) / (4. * (channels ** 2) * (size ** 2))

In [0]:
# an auxiliary loss function
# designed to maintain the "content" of the
# base image in the generated image
def content_loss(base, combination):
    channel_dim = 0 if K.image_dim_ordering() == "th" else -1

    try:
        channels = K.int_shape(base)[channel_dim]
    except TypeError:
        channels = K.shape(base)[channel_dim]
    size = input_width * input_height

    if content_loss_type == 1:
        multiplier = 1. / (2. * (channels ** 0.5) * (size ** 0.5))
    elif content_loss_type == 2:
        multiplier = 1. / (channels * size)
    else:
        multiplier = 1.

    return multiplier * K.sum(K.square(combination - base))

In [0]:
# the 3rd loss function, total variation loss,
# designed to keep the generated image locally coherent
def total_variation_loss(x):
    a = K.square(x[:, :input_width - 1, :input_height - 1] - x[:, 1:, :input_height - 1])
    b = K.square(x[:, :input_width - 1, :input_height - 1] - x[:, :input_width - 1, 1:])
    return K.sum(K.pow(a + b, 1.25))

In [0]:
def get_loss(outputs_dict, shape_dict, combination_image, nb_tensors):  
  # combine these loss functions into a single scalar
  loss = K.variable(0.)
  layer_features = outputs_dict[content_layer]  # 'conv5_2' or 'conv4_2'
  base_image_features = layer_features[0, :]
  combination_features = layer_features[nb_tensors - 1, :]
  loss = loss + content_weight * content_loss(base_image_features,
                                        combination_features)
  style_masks = [None] # If masks not present, pass None to the style loss

  channel_index = 1 if K.image_dim_ordering() == "th" else -1
  feature_layers = ['conv1', 'conv2', 'conv3', 'dense1', 'dense2', 'dense3']
  for layer_name in feature_layers:
      layer_features = outputs_dict[layer_name]
      shape = shape_dict[layer_name]
      combination_features = []
      style_reference_features = []
      if 'conv' in layer_name:
        combination_features = layer_features[nb_tensors - 1, :, :]
        style_reference_features = layer_features[1:nb_tensors - 1, :, :]
      else:
        combination_features = layer_features[nb_tensors - 1, :]
        style_reference_features = layer_features[1:nb_tensors - 1, :]
      sl = []
      sl.append(style_loss(style_reference_features[0], combination_features, style_masks[0], shape))
      loss = loss + (1 / len(feature_layers)) * sl[0]

  loss = loss + total_variation_weight * total_variation_loss(combination_image)
  return loss
  
def get_grad(loss, combination_image):
  # get the gradients of the generated image wrt the loss
  grads = K.gradients(loss, combination_image)
  outputs = [loss]
  if type(grads) in {list, tuple}:
      outputs += grads
  else:
      outputs.append(grads)
  return outputs

In [0]:
def eval_loss_and_grads(x, f_outputs):
    x = x.reshape((1, 257, 128))
    outs = f_outputs([x])
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values

In [0]:
# this Evaluator class makes it possible
# to compute loss and gradients in one pass
# while retrieving them via two separate functions,
# "loss" and "grads". This is done because scipy.optimize
# requires separate functions for loss and gradients,
# but computing them separately would be inefficient.
class Evaluator(object):
    def __init__(self, f_outputs):
        self.loss_value = None
        self.grads_values = None
        self.f_outputs = f_outputs

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x, self.f_outputs)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

# MIDI time
---



In [21]:
style_music = "MIDI-Unprocessed_SMF_02_R1_2004_01-05_ORIG_MID--AUDIO_02_R1_2004_05_Track05_wav.midi"

from google.colab import files
uploaded = files.upload()

In [22]:
content_music = "MIDI-Unprocessed_083_PIANO083_MID--AUDIO-split_07-09-17_Piano-e_2_-06_wav--5.midi"
from google.colab import files
uploaded = files.upload()

In [0]:
# for fn in uploaded.keys():
#   print('User uploaded file "{name}" with length {length} bytes'.format(
#       name=fn, length=len(uploaded[fn])))

In [0]:
def get_music(path):
  midi_pretty_format = pretty_midi.PrettyMIDI(path)
  piano_midi = midi_pretty_format.instruments[0] # Get the piano channels
  piano_roll = piano_midi.get_piano_roll(fs=5)
  piano_roll=piano_roll[:,:256]
  return piano_roll

In [0]:
def piano_roll_to_pretty_midi(piano_roll, fs=5, program=0):
    '''Convert a Piano Roll array into a PrettyMidi object
     with a single instrument.
    Parameters
    ----------
    piano_roll : np.ndarray, shape=(128,frames), dtype=int
        Piano roll of one instrument
    fs : int
        Sampling frequency of the columns, i.e. each column is spaced apart
        by ``1./fs`` seconds.
    program : int
        The program number of the instrument.
    Returns
    -------
    midi_object : pretty_midi.PrettyMIDI
        A pretty_midi.PrettyMIDI class instance describing
        the piano roll.
    '''
    notes, frames = piano_roll.shape
    pm = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=program)

    # pad 1 column of zeros so we can acknowledge inital and ending events
    piano_roll = np.pad(piano_roll, [(0, 0), (1, 1)], 'constant')

    # use changes in velocities to find note on / note off events
    velocity_changes = np.nonzero(np.diff(piano_roll).T)

    # keep track on velocities and note on times
    prev_velocities = np.zeros(notes, dtype=int)
    note_on_time = np.zeros(notes)

    for time, note in zip(*velocity_changes):
        # use time + 1 because of padding above
        velocity = piano_roll[note, time + 1]
        time = time / fs
        if velocity > 0:
            if prev_velocities[note] == 0:
                note_on_time[note] = time
                prev_velocities[note] = velocity
        else:
            pm_note = pretty_midi.Note(
                velocity=prev_velocities[note],
                pitch=note,
                start=note_on_time[note],
                end=time)
            instrument.notes.append(pm_note)
            prev_velocities[note] = 0
    pm.instruments.append(instrument)
    return pm

In [0]:
def deprocess_music(x):
    x = x.reshape((257, 128))
    notes = pianoroll_to_notes(x)
    return notes

In [27]:
# Create embedding layer to squeeze to dimensions to 16
c_music = get_music(content_music)
s_music = get_music(style_music)
print(c_music.shape)
print(s_music.shape)
print(c_music[:, :30].max(axis=0))
print(c_music[:, 5])

(128, 256)
(128, 256)
[ 0.  0.  0.  0.  0. 37.  0.  0.  0. 40. 40. 40. 40. 40.  0. 43. 43. 43.
 43. 38.  0.  0.  0.  0.  0.  0. 33.  0.  0.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0. 37.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.]


#Run Model 

In [28]:
input_tensor, shape, combination_image, nb_tensors = get_inputs(content_music, style_music)
vgg_model = get_custom_model(input_tensor, shape)
vgg_model.load_weights("model_best_1.h5")
vgg_model.summary()

ValueError: ignored

In [0]:
outputs_dict, shape_dict = get_output_dict_and_shape(vgg_model)
loss = get_loss(outputs_dict, shape_dict, combination_image, nb_tensors)
outputs = get_grad(loss, combination_image)
f_outputs = K.function([combination_image], outputs)
evaluator = Evaluator(f_outputs)

prev_min_val = -1

# x = preprocess_music_for_vgg(get_music(content_music))
x = midi_to_pianoroll(content_music)
x = x[:1,:,:]
improvement_threshold = float(min_improvement)

result_prefix = "midi_straight"

loss_aray1= []

for i in range(num_iter):
    print("Starting iteration %d of %d" % ((i + 1), num_iter))
    start_time = time.time()

    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(), fprime=evaluator.grads, maxfun=20)
    
    loss_aray1.append(min_val)
    if prev_min_val == -1:
        prev_min_val = min_val

    improvement = (prev_min_val - min_val) / prev_min_val * 100
    

    print('Current loss value:', min_val, " Improvement : %0.3f" % improvement, "%")
    prev_min_val = min_val
    # save current generated image
    img = deprocess_music(x.copy())

#     if not rescale_image:
#         img_ht = int(input_width * aspect_ratio)
#         print("Rescaling Image to (%d, %d)" % (input_width, img_ht))
#         img = imresize(img, (input_width, img_ht), interp=rescale_method)

#     if rescale_image:
#         print("Rescaling Image to (%d, %d)" % (input_WIDTH, input_HEIGHT))
#         img = imresize(img, (input_WIDTH, input_HEIGHT), interp=rescale_method)

    fname = result_prefix + '_at_iteration_%d.midi' % (i + 1)
    img = note_sequence_to_pretty_midi(img)
    img.write(fname)
    end_time = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i + 1, end_time - start_time))

    if improvement_threshold is not 0.0:
        if improvement < improvement_threshold and improvement is not 0.0:
            print("Improvement (%f) is less than improvement threshold (%f). Early stopping script." % (
                improvement, improvement_threshold))
            exit()

In [0]:
files.upload()

In [0]:
!ls

In [0]:
files.download("midi_straight_at_iteration_1.midi")

In [0]:
import matplotlib.pyplot as plt

plt.plot(loss_aray1)